- from dataset
- 연속형 데이터 타입 대상 스케일, 분할, 학습(train, test) 적용
- 배포는 컬럼을 4개 선별해 서비스와 결과 사람 인지하게 표시

In [17]:
# 1. 필요한 라이브러리 불러오기
import sklearn as sk
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import pickle

In [18]:
# 2. 데이터 불러오기
mh_prices = pd.read_csv('../../datasets/House Prices - Advanced Regression Techniques.csv')
mh_prices.head(3)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500


In [19]:
# 데이터 판다스에 담기

df_mh_prices = pd.DataFrame(mh_prices)
df_mh_prices

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [20]:
# 고유값의 수로 데이터 타입 구분하기
def category_columns(df_mh_prices):
    continuous_columns = []
    categorical_columns = []
    
    for column in df_mh_prices.columns :
        unique_values = df_mh_prices[column].nunique() # 고유값의 개수
        if unique_values < len(df_mh_prices) * 0.05 : # 고유값이 데이터의 5% 미만이면 범주형으로 간주
            categorical_columns.append(column)
        else:
            continuous_columns.append(column)
            
    return continuous_columns, categorical_columns

continuous, categorical = category_columns(df_mh_prices)


In [21]:
# 연속형 범주 확인
continuous

['Id',
 'LotFrontage',
 'LotArea',
 'YearBuilt',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'GrLivArea',
 'GarageYrBlt',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 'ScreenPorch',
 'SalePrice']

In [22]:
# 데이터 정보 확인
mh_prices[[
            'Id', 'LotFrontage', 'LotArea', 'YearBuilt', 'MasVnrArea'
           , 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF'
           , '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'GarageYrBlt', 'GarageArea'
           , 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', 'ScreenPorch','SalePrice'
           ]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   LotFrontage    1201 non-null   float64
 2   LotArea        1460 non-null   int64  
 3   YearBuilt      1460 non-null   int64  
 4   MasVnrArea     1452 non-null   float64
 5   BsmtFinSF1     1460 non-null   int64  
 6   BsmtFinSF2     1460 non-null   int64  
 7   BsmtUnfSF      1460 non-null   int64  
 8   TotalBsmtSF    1460 non-null   int64  
 9   1stFlrSF       1460 non-null   int64  
 10  2ndFlrSF       1460 non-null   int64  
 11  GrLivArea      1460 non-null   int64  
 12  GarageYrBlt    1379 non-null   float64
 13  GarageArea     1460 non-null   int64  
 14  WoodDeckSF     1460 non-null   int64  
 15  OpenPorchSF    1460 non-null   int64  
 16  EnclosedPorch  1460 non-null   int64  
 17  ScreenPorch    1460 non-null   int64  
 18  SalePric

In [23]:
# 결측치 확인
mh_prices[[
            'Id', 'LotFrontage', 'LotArea', 'YearBuilt', 'MasVnrArea'
           , 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF'
           , '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'GarageYrBlt', 'GarageArea'
           , 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', 'ScreenPorch','SalePrice'
           ]].isna().sum()

Id                 0
LotFrontage      259
LotArea            0
YearBuilt          0
MasVnrArea         8
BsmtFinSF1         0
BsmtFinSF2         0
BsmtUnfSF          0
TotalBsmtSF        0
1stFlrSF           0
2ndFlrSF           0
GrLivArea          0
GarageYrBlt       81
GarageArea         0
WoodDeckSF         0
OpenPorchSF        0
EnclosedPorch      0
ScreenPorch        0
SalePrice          0
dtype: int64

In [24]:
# 결측치 평균값으로 채우기
mh_prices['LotFrontage'] = mh_prices['LotFrontage'].fillna(mh_prices['LotFrontage'].mean())
mh_prices['MasVnrArea'] = mh_prices['MasVnrArea'].fillna(mh_prices['MasVnrArea'].mean())
mh_prices['GarageYrBlt'] = mh_prices['GarageYrBlt'].fillna(mh_prices['GarageYrBlt'].mean())

In [25]:
# 결측치 다시 확인
mh_prices.isnull().sum()

Id               0
MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
                ..
MoSold           0
YrSold           0
SaleType         0
SaleCondition    0
SalePrice        0
Length: 81, dtype: int64

In [26]:
# 1. 데이터 전처리 (스케일링)

# 연속형 데이터에 대해서만 스케일링
scaler = StandardScaler()

# 학습에 사용할 피쳐들만 선택하기 (연속형 데이터만 )
X = mh_prices[continuous].drop(columns = ['SalePrice'])

# 타겟 변수
y = mh_prices['SalePrice']

# 학습 데이터를 스케일링하기
X_scaled = scaler.fit_transform(X)

In [27]:
# 2. 훈련 데이터와 테스트 데이터 분리

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [28]:
# 3. 모델 학습 및 예측

# SVR 모델 학습
svr = SVR()
svr.fit(X_train, y_train)

# 예측
y_pred_svr = svr.predict(X_test)

# 평가
mean_squared_error(y_test, y_pred_svr)
r2_score(y_test, y_pred_svr)

-0.023999751361964528

In [29]:
# 선형 회귀 모델 학습
lr = LinearRegression()
lr.fit(X_train, y_train)

# 예측
y_pred_lr = lr.predict(X_test)

# 평가
mean_squared_error(y_test, y_pred_lr)
r2_score(y_test,y_pred_lr)

0.7754952900755876

In [30]:
# 랜덤 포레스트 회귀
rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)

# 예측
y_pred_rf = rf.predict(X_test)

# 평가
mean_squared_error(y_test,y_pred_rf)
r2_score(y_test, y_pred_rf)

0.858528929971167

In [ ]:
# 모델 저장(Pickle) 

save_file_name = '../../files/scalings_house-prices-advanced-regression-techniques.pkl'

